# Structuration DataSet (A ne plus relancer)

In [ ]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# On concaténe les 6 fichiers
import pandas as pd
df_0 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_0.csv")
df_1 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_1.csv")
df_2 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_2.csv")
df_3 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_3.csv")
df_4 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_4.csv")
df_5 = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/emaildata_100000_5.csv")
# df = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron.csv", nrows=1500)
frames = [df_0,df_1,df_2,df_3,df_4,df_5]
df = pd.concat(frames)
# la selection ci-dessous a permis de détecter 3 doublons (toutes colonnes identiques)
# df = df.loc[(df['subject'] == "Sagewood Town Homes")]
print(df.shape)
# df = df_emails.loc[(df['subject'] == "Sagewood Town Homes")]
df.drop('Unnamed: 0', axis=1, inplace=True)
# suppression des doublons (sur toutes les colonnes sauf index)
df.drop_duplicates(subset=['date','sender', 'recipient1', 'subject','text'],keep = 'first', inplace=True)
# on renomme les colonnes
df = df.rename(columns={'sender': 'from', 'recipient1': 'to','subject': 'header', 'text': 'body'})
# et export en csv du résultat.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique.csv"
df.to_csv(file_name, encoding='utf-8', index=False)
print(df.shape)
# Taille du fichier divisée par 2 environ
# génération d'un fichier light de 1000 lignes pour traitements plus rapides.
file_name = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_1000.csv"
df_emails.to_csv(file_name, encoding='utf-8', index=False)

Mounted at /content/drive
(405968, 6)
(227032, 5)


# Import - Data Preprocessing structures

Mount Gdrive files

In [1]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Import Spacy

In [2]:
#!pip install -U spacy
!python -m spacy download en_core_web_lg
# ligne ci-dessous a réexecuter ssi reinit du notebook
!pip install spacy-transformers

2022-07-27 14:57:58.208347: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 7.3 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
stopwords=list(STOP_WORDS)
from string import punctuation
punctuation=punctuation+ '\n'


In [4]:
import pandas as pd
# Pour limiter les temps de calcul on travaille uniquement sur les 'nrows' premiers mails
df_emails = pd.read_csv("/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_1000.csv", nrows=101)
df_emails.head()

,date,from,to,header,body
0,2001-05-14 16:39:00-07:00,phillip.allen@enron.com,tim.belden@enron.com,NaN,"['', 'Here is our forecast', '', ' ']"
1,2001-05-04 13:51:00-07:00,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"['', 'Traveling to have a business meeting tak..."
2,2000-10-18 03:00:00-07:00,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"['', 'test successful. way to go!!!']"
3,2000-10-23 06:13:00-07:00,phillip.allen@enron.com,randall.gay@enron.com,NaN,"['', 'Randy,', '', ' Can you send me a schedul..."
4,2000-08-31 05:07:00-07:00,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"['', ""Let's shoot for Tuesday at 11:45. ""]"


Overview dataset

In [5]:
# nbre d'expéditeurs distincts
len(df_emails['from'].unique())

1

In [6]:
# nbre de destinataires distincts
len(df_emails['to'].unique())

49

Rajout colonnes pour traitements ultérieurs

In [7]:
import numpy as np
df_emails = df_emails.replace('NaN', np.NaN)
# df_emails = df[df['header'].notna()]

df_emails['cc'] = ''                        # pour identfication des personnes en copie
df_emails['NER_header'] = ''                # Spacy NER
df_emails['NER_body'] = ''                  # Spacy NER
df_emails['body_clean'] = ''                # body sans caracteres parasites
df_emails['body_dict'] = ''                 # body tokenisé
df_emails['summary_TFIDF'] = ''             # extractive summary by TF-IDF
df_emails['summary_TFIDF_sim'] = ''         # calcul de similarité par rapport à body_clean
df_emails['summary_spacy'] = ''             # extractive summary by Spacy
df_emails['summary_spacy_sim'] = ''         # calcul de similarité par rapport à body_clean
df_emails['summary_BART'] = ''              # abstractive summary by HugginFace/BART
df_emails['summary_BART_sim'] = ''          # calcul de similarité par rapport à body_clean
df_emails['summary_T5'] = ''                # abstractive summary by HugginFace/T5
df_emails['summary_T5_sim'] = ''            # calcul de similarité par rapport à body_clean
df_emails['best_sim'] = ''                  # meilleure taux de similarité par rapport à body_clean
df_emails.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date               101 non-null    object
 1   from               101 non-null    object
 2   to                 101 non-null    object
 3   header             61 non-null     object
 4   body               101 non-null    object
 5   cc                 101 non-null    object
 6   NER_header         101 non-null    object
 7   NER_body           101 non-null    object
 8   body_clean         101 non-null    object
 9   body_dict          101 non-null    object
 10  summary_TFIDF      101 non-null    object
 11  summary_TFIDF_sim  101 non-null    object
 12  summary_spacy      101 non-null    object
 13  summary_spacy_sim  101 non-null    object
 14  summary_BART       101 non-null    object
 15  summary_BART_sim   101 non-null    object
 16  summary_T5         101 non-null    object
 1

In [8]:
#
# A ce stade le fichier a la structure nécessaire à Melusine pour le nettoyage
#

# NER with Spacy

In [9]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_lg')
spacy_stopwords = nlp.Defaults.stop_words
stopwords = spacy_stopwords
print(spacy_stopwords)


deb = 0
fin = df_emails.shape[0]
for x in range(deb,fin):
# subject
  subj = df_emails.iloc[x]['header']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_header'][x] = doc_subj.ents

# body
  subj = df_emails.iloc[x]['body']
#  print(x,' - ',subj)
  doc_subj = nlp(str(subj))
  df_emails['NER_body'][x] = doc_subj.ents

{'call', 'themselves', 'become', 'only', 'bottom', 'whence', 'their', 'or', 'wherever', 'meanwhile', 'latterly', 'further', 'else', 'anyhow', 'until', 'toward', 'whenever', 'whither', 'will', 'her', '‘ve', 'also', 'each', 'thus', 'should', '’ve', 'often', 'former', 'them', 'what', 'under', 'around', 'thereafter', 'give', 'seems', 'he', 'per', 'doing', "'s", 'move', 'into', 'hereafter', 'together', "'ll", 'too', 'everything', 'from', 'though', 'already', 'be', 'no', 'being', 'quite', '‘re', 'that', 'put', 'otherwise', 'whereupon', 'towards', "n't", 'could', 'beyond', 'through', 'n‘t', 'please', 'it', 'across', 'n’t', 'while', 'ourselves', 'nothing', 'amongst', 'name', 'we', 'any', 'noone', 'ca', 'not', 'whereafter', 'few', 'alone', 'therein', 'another', 'used', 'fifteen', 'perhaps', 'well', 'somehow', 'except', 'whoever', 'yourself', 'by', 'someone', 'sometime', 'anyway', 'keep', 'without', 'was', 'next', 'many', 'due', '’re', 'does', 'get', 'really', 'and', 'therefore', 'him', 'but', '

In [10]:
df_emails[['header','NER_header','NER_body']][0:40]

,header,NER_header,NER_body
0,NaN,(),()
1,Re:,(),"((Austin),)"
2,Re: test,(),()
3,NaN,(),"((Randy), (Patti, S), (Phillip))"
4,Re: Hello,(),"((Tuesday),)"
5,Re: Hello,(),"((Greg), (next, Tuesday), (Thursday), (Phillip))"
6,NaN,(),"((Phillip, Allen), (Mike, Grigsby), (Keith, Ho..."
7,Re: PRC review - phone calls,"((PRC),)","((between, 10, and, 11:30),)"
8,Re: High Speed Internet Access,(),"((2), (3))"
9,FW: fixed forward or other Collar floor gas pr...,(),"((Phillip, K, Allen, /, HOU), (10/16/2000), (0..."


In [11]:
id = 12
doc= nlp(df_emails['header'][id])
displacy.render(doc, style = "ent",jupyter = True)
print("----- eMail ",id," -----")
doc= nlp(df_emails['body'][id])
displacy.render(doc, style = "ent",jupyter = True)
print("\n")

/usr/local/lib/python3.7/dist-packages/spacy/displacy/__init__.py:205: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


----- eMail  12  -----


In [12]:
df_emails[['header','body','body_dict','summary_TFIDF','NER_body']][0:30]

,header,body,body_dict,summary_TFIDF,NER_body
0,NaN,"['', 'Here is our forecast', '', ' ']",,,()
1,Re:,"['', 'Traveling to have a business meeting tak...",,,"((Austin),)"
2,Re: test,"['', 'test successful. way to go!!!']",,,()
3,NaN,"['', 'Randy,', '', ' Can you send me a schedul...",,,"((Randy), (Patti, S), (Phillip))"
4,Re: Hello,"['', ""Let's shoot for Tuesday at 11:45. ""]",,,"((Tuesday),)"
5,Re: Hello,"['', 'Greg,', '', ' How about either next Tues...",,,"((Greg), (next, Tuesday), (Thursday), (Phillip))"
6,NaN,"['', 'Please cc the following distribution lis...",,,"((Phillip, Allen), (Mike, Grigsby), (Keith, Ho..."
7,Re: PRC review - phone calls,"['', 'any morning between 10 and 11:30']",,,"((between, 10, and, 11:30),)"
8,Re: High Speed Internet Access,"['', '1. login: pallen pw: ke9davis', '', "" I...",,,"((2), (3))"
9,FW: fixed forward or other Collar floor gas pr...,"['', '---------------------- Forwarded by Phil...",,,"((Phillip, K, Allen, /, HOU), (10/16/2000), (0..."


In [13]:
# import spacy
# from spacy import displacy
# nlp = spacy.load('en_core_web_sm')
# spacy_stopwords = nlp.Defaults.stop_words
# stopwords = spacy_stopwords
# print(spacy_stopwords)

# Data cleaning

In [14]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import re

def data_clean (tokens):
# Suppression des caractères spéciaux ou inutiles \“‘,]
#  print(“Data cleaning --\n”)
#  print(“Avant :\n”,type(tokens),” “,tokens)
   r = re.compile(r"[\\\''\'\"\[\]]+")
   tok=r.sub('', tokens)
   r = re.compile(r"([\(\)]{1,}|,{2,})")
   tok2=r.sub(' ', tok)
   r = re.compile(r"((\.\s?,)+|(, ,)|(^,{1,}))")
   tok3=r.sub('', tok2)
   r = re.compile(r"\s{2,}")
   texte=r.sub(' ', tok3)
#  texte = sent_tokenize(texte)  # découpe en phrases stockées dans une liste
#  print(“Après sent_tokenize :\n”,type(texte),” “,texte)
   return texte

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
def remove_stop_words(text,stopwords):
  lst=[]
#  for token in str(text).split():
  for token in text.split():
    if token.lower() not in stopwords:    #checking whether the word is not 
      lst.append(token)                    #present in the stopword list.      
#    else:
#      print("suppression du stopword : ",token)
  # Join items in the list
#   print("Original text before removing stopwords  :   \n",text)
  result = ' '.join(lst)
#   print("Text after removing stopwords  :   \n",result)
  return result

In [16]:
# Transforme Text en Dictionnaire - Key = indice de phrase - Value = Mots de la phrase
import re
from nltk.tokenize import sent_tokenize,word_tokenize

def dictionarize(article):
    dico={}
    phrases=sent_tokenize(article)
    phr2=[]
    for i, phr in enumerate(phrases):
        import re
        phr=re.sub(pattern ="[^a-zA-Z]", repl = ' ', string = phr)
        phr2.append(phr)
    
        phr=word_tokenize(phr)
        phr_lower=[w.lower() for w in phr]
        if len(phr_lower)>4:      
            dico[i]=phr_lower    
    return dico



In [17]:
df_emails.shape[0]

101

# Summary methodS

In [18]:
import pandas as pd
import numpy as np

def TF(token, artikle):
    """
    Calcule le score TF d'un mot dans un artikle
    
    token : Mot dont le score TF doit être calculé.
    
    artikle : Dictionnaire généré à partir d'un texte.
    """
    f = 0
    for key in artikle:
        for work in artikle[key]:
            if work == token:
                f += 1 
    return np.log(f+1)

def IDF(token, corpus):
    """
    Calcule le score IDF d'un mot dans un corpus d'artikles.
    
    token : Mot dont le score IDF doit être calculé.
    
    corpus : Liste d'artikles.
    """
    N = len(corpus)
    d=0
    present = False
    
    for artikle in corpus:
        for key in artikle:
            if token in artikle[key]:
                present = True
        d += int(present)
        present = False
                
    return np.log(N/(d+1) +1)

def TFIDF(token, artikle, corpus):
    """
    Calcule le score TF-IDF d'un mot dans un texte.
    
    token : mot dont le score doit être calculé.
    
    artikle : artikle qui servira à calculer le score du mot dans cet artikle.
    
    corpus : Liste d'artikles formant le corpus.
    """
    return TF(token, artikle)*IDF(token, corpus)

def score_sentence(corpus, artikle, sentence):
    """
    Calcule le score d'une phrase.
    
    corpus : Liste d'artikles.
    
    artikle : Dictionnaire de phrases.
    
    sentence : Phrase sous forme de liste de mots.
    """
    score_sentence = []
    for word in sentence :
        score_word = TFIDF(word, artikle, corpus)
        score_sentence.append(score_word)
    return np.mean(score_sentence)

def best_sentences(scores_artikle, nb_sentences):
    """
    Retourne les indices des phrases les plus importantes en fonction des scores obtenus.
    
    scores_artikle : Liste des scores de chaque phrase dans un texte.
    
    nb_sentences : Nombre de phrases à sélectionner.
    """
    
    return sorted(np.argsort(scores_artikle)[-nb_sentences:])

def summarize_TFIDF(i, n_sentences, df):
    """
    Synthèse extractive d'un article par la méthode TF-IDF.
    
    i : indice de l'article dans le corpus.
    
    n_sentences : nombre de phrases à conserver.
    
    df : DataFrame contenant les artikles dans une colonne 'Artikle'.    
    """
    corpus = df['body_dict']
    artikle = corpus[i]
    texte=df['body'][i]
    
    if len(corpus) <= n_sentences:
      print("Longeur corpus inférieure au nombre minimal de phrases retenu pour le résumé")
      return texte
    # Calcul du score de chaque phrase de l'artikle
    scores_artikle = [score_sentence(corpus, artikle, sentence) for sentence in artikle.values()]
 
    
    # Extraction des indices des phrases ayant les meilleurs scores
    result = best_sentences(scores_artikle, n_sentences)
#    print('best sentences :',result)
    
    # Séparation de phrases l'article original 
    tokenized_article = sent_tokenize(texte)
    
    # Extraction des phrases les plus importantes
    summary = [tokenized_article[i] for i in result]
#    print("[",i,"] :",summary)
    # transformation finale en chaine de caracteres
    texte = ''.join(summary)
    return texte

def summarize_spacy (text,ratio):
  # https://www.numpyninja.com/post/text-summarization-through-use-of-spacy-library
  # print('\n Texte à résumer avec un ratio de ',ratio*100,"%")
  # print(text)
  nlp = spacy.load('en_core_web_lg')
  doc= nlp(text)
  tokens=[token.text for token in doc]
#  print("Tokens : \n",tokens)

  # calcul frequence de mots
  word_frequencies={}
  for word in doc:
    if word.text.lower() not in stopwords:
      if word.text.lower() not in punctuation:
        if word.text not in word_frequencies.keys():
          word_frequencies[word.text] = 1
        else:
          word_frequencies[word.text] += 1
  # print("\n word_frequencies : ",word_frequencies)

  # normalisation des frequence de mots
  max_frequency=max(word_frequencies.values())
  # print('Max Frequency : ',max_frequency)
  for word in word_frequencies.keys():
    word_frequencies[word]=word_frequencies[word]/max_frequency
  # print("\n Normalized word_frequencies : \n",word_frequencies)
  # Sentences token
  sentence_tokens= [sent for sent in doc.sents]
  # print("sentence_tokens ",sentence_tokens)
  # Calculate the most important sentences by adding the word frequencies in each sentence.
  sentence_scores = {}
  for sent in sentence_tokens:
    for word in sent:
      if word.text.lower() in word_frequencies.keys():
        if sent not in sentence_scores.keys():                            
          sentence_scores[sent]=word_frequencies[word.text.lower()]
        else:
          sentence_scores[sent]+=word_frequencies[word.text.lower()]

  # identifier % (ratio) du texte avec score maximum
  from heapq import nlargest
  select_length=int(len(sentence_tokens)*ratio)
  # print('select_length ',select_length)
  if select_length != 0:
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
  else:
    summary = 'See Original'
  return summary



In [19]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
!pip install datasets transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:

from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
my_model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

def summarize_T5 (text):
  text = "summarize:" + text
  # encoding the input text
  input_ids=tokenizer.encode(text, return_tensors='pt', max_length=512)
  # Generating summary ids
  summary_ids = my_model.generate(input_ids)
  # Decoding the tensor and printing the summary.
  t5_summary = tokenizer.decode(summary_ids[0])
  return t5_summary
  



/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [22]:
def summarize_BART (text):
  print("Texte a résumer",text)
  from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
  # Loading the model and tokenizer for bart-large-cnn
  tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
  model=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
  # Encoding the inputs and passing them to model.generate()
  inputs = tokenizer.batch_encode_plus([text],return_tensors='pt')
  summary_ids = model.generate(inputs['input_ids'], early_stopping=True)
  # Decoding and printing the summary
  bart_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  print('bart_summary',bart_summary)
  return bart_summary


# Test Unitaire

TF-IDF

In [23]:
id = 44
df_emails['body_clean'][id] = data_clean(df_emails['body'].iloc[id])

text = df_emails['body_clean'][id]
print("Original text : ")
print(df_emails['body'][id])
print(df_emails['body_dict'][id])
print("\nCleaned text : ",type(text),len(text)," car. : ")
print(text,"\n")
# # EXTRACTIVE summary using TF-IDF
# ------------------------------------------------------------------------
# suppression de stopwords pour TFIDF
text = remove_stop_words(text,stopwords)

# Dictionnarize the text to be used with TF-IDF
print("Extractive summary using TF-IDF")
max_sent = 3  # Pas de summary si moins de 4 lignes.
df_emails['body_dict'][id] = dictionarize(text)   # sans stopwords
if len(df_emails['body_dict'][id]) > max_sent:
  text = df_emails['body_dict'][id]
  # on affiche phrase par phrase le contenu de chaque mail
  for cle, valeur in text.items():
    print("Ligne ", cle, " : ", valeur)
  # Identification des max_sent les plus significatives 
  df_emails['summary_TFIDF'][id] = summarize_TFIDF(id, max_sent, df_emails)
  ratio = len(df_emails['summary_TFIDF'][id])/len(df_emails['body_clean'][id])
  print("\nRésumé TFIDF - Len ",len(df_emails['summary_TFIDF'][id])," Ratio : ",int(ratio*100),"%")
  print(df_emails['summary_TFIDF'][id])
else:
  print("ID ",id," contenu trop court : ",len(df_emails['body_dict'][id])," lignes - Reprise de l'original pour TFIDF")
  df_emails['summary_TFIDF'][id] = df_emails['body_clean'][id]


Original text : 
['', 'Jeff,', '', ' I need to see the site plan for Burnet.  Remember I must get written ', 'approval from Brenda Key Stone before I can sell this property and she has ', 'concerns about the way the property will be subdivided.    I would also like ', 'to review the closing statements as soon as possible.', '', 'Phillip']


Cleaned text :  <class 'str'> 293  car. : 
 Jeff , I need to see the site plan for Burnet. Remember I must get written , approval from Brenda Key Stone before I can sell this property and she has , concerns about the way the property will be subdivided. I would also like , to review the closing statements as soon as possible , Phillip 

Extractive summary using TF-IDF
ID  44  contenu trop court :  3  lignes - Reprise de l'original pour TFIDF


In [24]:
df_emails['body_dict'][id]

{0: ['jeff', 'need', 'site', 'plan', 'burnet'],
 1: ['remember',
  'written',
  'approval',
  'brenda',
  'key',
  'stone',
  'sell',
  'property',
  'concerns',
  'way',
  'property',
  'subdivided'],
 2: ['like', 'review', 'closing', 'statements', 'soon', 'possible', 'phillip']}

In [25]:
id

44

Spacy

In [26]:
# ------------------------------------------------------------------------
# EXTRACTIVE summary using Spacy
print("\nExtractive summary using Spacy")
ratio = 0.25  # 25% du nombre initial de lignes
text = df_emails['body_clean'].iloc[id]
df_emails['summary_spacy'][id] = summarize_spacy (str(text),ratio)
ratio = len(df_emails['summary_spacy'][id])/len(df_emails['body_clean'][id])
print("Résumé Spacy - Len ",len(df_emails['summary_spacy'][id]),"Ratio : ",int(ratio*100),"%")
print(df_emails['summary_spacy'][id])



Extractive summary using Spacy
Résumé Spacy - Len  12 Ratio :  4 %
See Original


Transformers : T5

In [27]:
df_emails['summary_T5'] = ''

In [28]:
id=44
print("\nExtractive summary using Transformers / T5")
text = df_emails['body_clean'].iloc[id]
print('text :',text)
print(len(df_emails['body_dict'].iloc[id]))
if len(df_emails['body_dict'].iloc[id]) !=0:
  try:
    df_emails['summary_T5'][id] = summarize_T5 (text)
    ratio = len(df_emails['summary_T5'][id])/len(df_emails['body_clean'][id])
    print("\nRésumé Transformers T5  - Len ",len(df_emails['summary_T5'][id])," Ratio : ",int(ratio*100),"%")
  except:
    print("Pb extractive summary on Id ",id)
    df_emails['summary_T5'][id] = "See Original"

print(df_emails['summary_T5'][id])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Extractive summary using Transformers / T5
text :  Jeff , I need to see the site plan for Burnet. Remember I must get written , approval from Brenda Key Stone before I can sell this property and she has , concerns about the way the property will be subdivided. I would also like , to review the closing statements as soon as possible , Phillip
3

Résumé Transformers T5  - Len  65  Ratio :  22 %
<pad> i need to see the site plan for Burnet. I must get written,


Transformers : BART

In [29]:
# ------------------------------------------------------------------------
# Extractive summary using Transformers
# BART model

print("\nExtractive summary using Transformers / BART")
text = df_emails['body_clean'].iloc[id]
try:
  df_emails['summary_BART'][id] = summarize_BART (text)
  ratio = len(df_emails['summary_abstractive'][id])/len(df_emails['body_clean'][id])
  print("\nRésumé Transformers BART  - Len ",len(df_emails['summary_abstractive'][id])," Ratio : ",int(ratio*100),"%")
except:
  print("Pb extractive summary on Id ",id)
  df_emails['summary_BART'][id] = "Erreur"

print(df_emails['summary_BART'][id])


Extractive summary using Transformers / BART
Texte a résumer  Jeff , I need to see the site plan for Burnet. Remember I must get written , approval from Brenda Key Stone before I can sell this property and she has , concerns about the way the property will be subdivided. I would also like , to review the closing statements as soon as possible , Phillip
bart_summary I need to see the site plan for Burnet. Remember I must get written, approval from Brenda Key Stone before I can sell this property. I would also like, to review the closing statements as soon as possible, Phillip. Phillip is the owner of the property in Burnet, Texas.
Pb extractive summary on Id  44
Erreur


# Boucle de traitement

In [30]:
deb = 0
fin = df_emails.shape[0]
#fin = 50
# fin = deb
print("Traitement en cours pour ",fin, "records sauf BART model")
for id in range(deb,fin):
  print("\n id : ",id)
  print('--------------------')
  # coller quand OK pour les tests unitaires
  df_emails['body_clean'][id] = data_clean(str(df_emails['body'].iloc[id]))
  text = df_emails['body_clean'][id]
  print("Cleaned text : ",len(str(text))," car.\n")
  print(text)
  # Extractive summary using TF-IDF
  # ------------------------------------------------------------------------
  # suppression de stopwords pour TFIDF
  text = remove_stop_words(text,stopwords)

  # Dictionnarize the text to be used with TF-IDF
  df_emails['body_dict'][id] = dictionarize(text)   # sans stopwords
  max_sent = 3 
  if len(df_emails['body_dict'][id]) > max_sent:
    text = df_emails['body_dict'][id]
    # on affiche phrase par phrase le contenu de chaque mail
    for cle, valeur in text.items():
      print("Ligne ", cle, " : ", valeur)
    # Identification des n_sent les plus significatives

    df_emails['summary_TFIDF'][id] = summarize_TFIDF(id, max_sent, df_emails)
    ratio = len(df_emails['summary_TFIDF'][id])/len(df_emails['body_clean'][id])
    print("\nRésumé TFIDF - Len ",len(df_emails['summary_TFIDF'][id])," Ratio : ",int(ratio*100),"%")
    print(df_emails['summary_TFIDF'][id])
  else:
    print("ID ",id," contenu trop court. Reprise de l'original pour TFIDF")
    df_emails['summary_TFIDF'][id] = df_emails['body_clean'][id]

# ------------------------------------------------------------------------
  # Extractive summary using Spacy
  ratio = 0.20
  text = df_emails['body_clean'].iloc[id]
  df_emails['summary_spacy'][id] = summarize_spacy (str(text),ratio)
  ratio = len(df_emails['summary_spacy'][id])/len(df_emails['body_clean'][id])
  print("\nRésumé Spacy - Len ",len(df_emails['summary_spacy'][id]),"Ratio : ",int(ratio*100),"%")
  print(df_emails['summary_spacy'][id])

# ------------------------------------------------------------------------
# Abstractive summary using Transformers T5
  print("\nExtractive summary using Transformers / T5")
  text = df_emails['body_clean'].iloc[id]
  print('text :',text)
  if len(df_emails['body_dict'].iloc[id]) !=0:
    try:
      df_emails['summary_T5'][id] = summarize_T5 (text)
      ratio = len(df_emails['summary_T5'][id])/len(df_emails['body_clean'][id])
      print("\nRésumé Transformers T5  - Len ",len(df_emails['summary_T5'][id])," Ratio : ",int(ratio*100),"%")
    except:
      print("Pb extractive summary on Id ",id)
      df_emails['summary_T5'][id] = "See Original"


Traitement en cours pour  101 records sauf BART model

 id :  0
--------------------
Cleaned text :  22  car.

 Here is our forecast 
ID  0  contenu trop court. Reprise de l'original pour TFIDF

Résumé Spacy - Len  12 Ratio :  54 %
See Original

Extractive summary using Transformers / T5
text :  Here is our forecast 

 id :  1
--------------------
Cleaned text :  774  car.

 Traveling to have a business meeting takes the fun out of the trip. Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings. I would even try and get some honest opinions on whether a trip is even desired or necessary , As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not. Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be better if held in a ro

In [31]:
print("Traitement en cours pour ",fin, "records avec BART model.")
for id in range(deb,fin):
  print("\n id : ",id)
  print('--------------------')
  # coller quand OK pour les tests unitaires
  df_emails['body_clean'][id] = data_clean(str(df_emails['body'].iloc[id]))
  text = df_emails['body_clean'][id]
  print("Cleaned text : ",len(str(text))," car.\n")
  print(text)
 
# ------------------------------------------------------------------------
# Abstractive summary using Transformers BART
  print("\nExtractive summary using Transformers / BART")
  text = df_emails['body_clean'].iloc[id]
  print('text :',text)
  try:
    df_emails['summary_BART'][id] = summarize_BART (text)
    ratio = len(df_emails['summary_BART'][id])/len(df_emails['body_clean'][id])
    print("\nRésumé Transformers BART  - Len ",len(df_emails['summary_BART'][id])," Ratio : ",int(ratio*100),"%")
  except:
    print("Pb extractive summary on Id ",id)
    df_emails['summary_BART'][id] = "Erreur"

  print(df_emails['summary_BART'][id])

Traitement en cours pour  101 records avec BART model.

 id :  0
--------------------
Cleaned text :  22  car.

 Here is our forecast 

Extractive summary using Transformers / BART
text :  Here is our forecast 
Texte a résumer  Here is our forecast 
bart_summary Here is our forecast for the rest of the year. Here is our prediction for the next few months. Here are our predictions for the coming year. We will provide updates on our website and social media throughout the season. Visit CNN.com/Weather for the latest weather updates.

Résumé Transformers BART  - Len  273  Ratio :  1240 %
Here is our forecast for the rest of the year. Here is our prediction for the next few months. Here are our predictions for the coming year. We will provide updates on our website and social media throughout the season. Visit CNN.com/Weather for the latest weather updates.

 id :  1
--------------------
Cleaned text :  774  car.

 Traveling to have a business meeting takes the fun out of the trip. Especia

Token indices sequence length is longer than the specified maximum sequence length for this model (1179 > 1024). Running this sequence through the model will result in indexing errors


Pb extractive summary on Id  12
Erreur

 id :  13
--------------------
Cleaned text :  5681  car.

 ---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/09/2000 , 02:00 PM --------------------------- , Richard Burchfield, 10/06/2000 06:59 AM, To: Phillip K Allen/HOU/ECT@ECT, cc: Beth Perlman/HOU/ECT@ECT , Subject: Consolidated positions: Issues & To Do list Phillip Below is the issues & to do list as we go forward with documenting the , requirements for consolidated physical/financial positions and transport , trade capture. What we need to focus on is the first bullet in Allans list; , the need for a single set of requirements. Although the meeting with Keith on Wednesday, was informative the solution of creating a infinitely dynamic , consolidated position screen, will be extremely difficult and time , consuming. Throughout the meeting on Wednesday, Keith alluded to the , inability to get consensus amongst the traders on the presentation of the , consolidated position, s

Token indices sequence length is longer than the specified maximum sequence length for this model (1179 > 1024). Running this sequence through the model will result in indexing errors


Pb extractive summary on Id  13
Erreur

 id :  14
--------------------
Cleaned text :  113  car.

 Dave , Here are the names of the west desk members by category. The origination , side is very sparse. Phillip, 

Extractive summary using Transformers / BART
text :  Dave , Here are the names of the west desk members by category. The origination , side is very sparse. Phillip, 
Texte a résumer  Dave , Here are the names of the west desk members by category. The origination , side is very sparse. Phillip, 
bart_summary Here are the names of the west desk members by category. The origination, side is very sparse. Phillip,  Phillip, Phillip, Dave, Dave, Dave, David, Phillip. Phillip and Dave, Phillip and Phillip, David and Phillip. Philip and Phillip were the first two to join the West Desk.

Résumé Transformers BART  - Len  275  Ratio :  243 %
Here are the names of the west desk members by category. The origination, side is very sparse. Phillip,  Phillip, Phillip, Dave, Dave, Dave, David, 

...

Pb extractive summary on Id  57
Erreur

 id :  58
--------------------
Cleaned text :  513  car.

 ---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/06/2000 , 03:53 PM --------------------------- , Lucy Gonzalez <stagecoachmama@hotmail.com> on 09/06/2000 09:06:45 AM, To: pallen@enron.com, cc: , Subject: utilities roll _________________________________________________________________________, Get Your Private, Free E-mail from MSN Hotmail at http://www.hotmail.com , Share information about yourself, create your own public profile at, http://profiles.msn.com , - utility.xls, - utility.xls, 

Extractive summary using Transformers / BART
text :  ---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/06/2000 , 03:53 PM --------------------------- , Lucy Gonzalez <stagecoachmama@hotmail.com> on 09/06/2000 09:06:45 AM, To: pallen@enron.com, cc: , Subject: utilities roll _________________________________________________________________________, Get Your Private, Free

Token indices sequence length is longer than the specified maximum sequence length for this model (2089 > 1024). Running this sequence through the model will result in indexing errors


Pb extractive summary on Id  61
Erreur

 id :  62
--------------------
Cleaned text :  60  car.

 Ina , I scheduled a meeting with Jean Mrha tomorrow at 3:30

Extractive summary using Transformers / BART
text :  Ina , I scheduled a meeting with Jean Mrha tomorrow at 3:30
Texte a résumer  Ina , I scheduled a meeting with Jean Mrha tomorrow at 3:30
bart_summary "I scheduled a meeting with Jean Mrha tomorrow at 3:30," Ina said. "I'm looking forward to working with him," she added. "He's a great guy," InA said. 'He's always been very supportive of me. He's a good guy'

Résumé Transformers BART  - Len  208  Ratio :  346 %
"I scheduled a meeting with Jean Mrha tomorrow at 3:30," Ina said. "I'm looking forward to working with him," she added. "He's a great guy," InA said. 'He's always been very supportive of me. He's a good guy'

 id :  63
--------------------
Cleaned text :  397  car.

 Ina , Can you pull Tori K.s and Martin Cuillas resumes and past performance , reviews from H.R , ---------

Token indices sequence length is longer than the specified maximum sequence length for this model (1897 > 1024). Running this sequence through the model will result in indexing errors


Pb extractive summary on Id  71
Erreur

 id :  72
--------------------
Cleaned text :  151  car.

 John Lavorato-M Mike Grigsby-D, Keith Holst-D, Frank Ermis-D, Steve South-D, Janie Tholt-D Scott Neal-P, Hunter Shively-P, Tom Martin-P, John Arnold-P

Extractive summary using Transformers / BART
text :  John Lavorato-M Mike Grigsby-D, Keith Holst-D, Frank Ermis-D, Steve South-D, Janie Tholt-D Scott Neal-P, Hunter Shively-P, Tom Martin-P, John Arnold-P
Texte a résumer  John Lavorato-M Mike Grigsby-D, Keith Holst-D, Frank Ermis-D, Steve South-D, Janie Tholt-D Scott Neal-P, Hunter Shively-P, Tom Martin-P, John Arnold-P
bart_summary John Lavorato, Mike Grigsby, Keith Holst, Frank Ermis, Steve South, Janie Tholt-D, Scott Neal-P, Hunter Shively-P. John Arnold, Tom Martin-P and John Arnold-P are the referees.

Résumé Transformers BART  - Len  176  Ratio :  116 %
John Lavorato, Mike Grigsby, Keith Holst, Frank Ermis, Steve South, Janie Tholt-D, Scott Neal-P, Hunter Shively-P. John Arnold, Tom M

In [32]:
# doc1.similarity(doc2)
#!python -m spacy download en_core_web_lg

In [33]:
df_emails.head(20)

,date,from,to,header,body,cc,NER_header,NER_body,body_clean,body_dict,summary_TFIDF,summary_TFIDF_sim,summary_spacy,summary_spacy_sim,summary_BART,summary_BART_sim,summary_T5,summary_T5_sim,best_sim
0,2001-05-14 16:39:00-07:00,phillip.allen@enron.com,tim.belden@enron.com,NaN,"['', 'Here is our forecast', '', ' ']",,(),(),Here is our forecast,{},Here is our forecast,,See Original,,Here is our forecast for the rest of the year....,,,,
1,2001-05-04 13:51:00-07:00,phillip.allen@enron.com,john.lavorato@enron.com,Re:,"['', 'Traveling to have a business meeting tak...",,(),"((Austin),)",Traveling to have a business meeting takes th...,"{0: ['traveling', 'business', 'meeting', 'take...","['', 'Traveling to have a business meeting tak...",,I would even try and get some honest opinions ...,,Traveling to have a business meeting takes the...,,"<pad> if you have to prepare a presentation, y...",,
2,2000-10-18 03:00:00-07:00,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,"['', 'test successful. way to go!!!']",,(),(),test successful. way to go!!!,{},test successful. way to go!!!,,See Original,,Test successful. way to go!!! way to going!!!....,,,,
3,2000-10-23 06:13:00-07:00,phillip.allen@enron.com,randall.gay@enron.com,NaN,"['', 'Randy,', '', ' Can you send me a schedul...",,(),"((Randy), (Patti, S), (Phillip))","Randy , Can you send me a schedule of the sal...","{0: ['randy', 'send', 'schedule', 'salary', 'l...","Randy , Can you send me a schedule of the sal...",,See Original,,"Randy, Can you send me a schedule of the salar...",,"<pad> Randy, can you send me a schedule of the...",,
4,2000-08-31 05:07:00-07:00,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"['', ""Let's shoot for Tuesday at 11:45. ""]",,(),"((Tuesday),)",Lets shoot for Tuesday at 11:45.,{},Lets shoot for Tuesday at 11:45.,,See Original,,Lets shoot for Tuesday at 11:45. Let's shoot f...,,,,
5,2000-08-31 04:17:00-07:00,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,"['', 'Greg,', '', ' How about either next Tues...",,(),"((Greg), (next, Tuesday), (Thursday), (Phillip))","Greg , How about either next Tuesday or Thurs...",{},"Greg , How about either next Tuesday or Thurs...",,See Original,,"Greg, How about either next Tuesday or Thursda...",,,,
6,2000-08-22 07:44:00-07:00,phillip.allen@enron.com,david.l.johnson@enron.com,NaN,"['', 'Please cc the following distribution lis...",,(),"((Phillip, Allen), (Mike, Grigsby), (Keith, Ho...",Please cc the following distribution list wit...,"{0: ['cc', 'following', 'distribution', 'list'...",Please cc the following distribution list wit...,,See Original,,Please cc the following distribution list with...,,<pad> cc the following distribution list with ...,,
7,2000-07-14 06:59:00-07:00,phillip.allen@enron.com,joyce.teixeira@enron.com,Re: PRC review - phone calls,"['', 'any morning between 10 and 11:30']",,"((PRC),)","((between, 10, and, 11:30),)",any morning between 10 and 11:30,{},any morning between 10 and 11:30,,See Original,,CNN.com will feature iReporter photos in a wee...,,,,
8,2000-10-17 02:26:00-07:00,phillip.allen@enron.com,mark.scott@enron.com,Re: High Speed Internet Access,"['', '1. login: pallen pw: ke9davis', '', "" I...",,(),"((2), (3))",1. login: pallen pw: ke9davis I dont think th...,"{0: ['login', 'pallen', 'pw', 'ke', 'davis', '...",1. login: pallen pw: ke9davis I dont think th...,,"static IP address IP: 64.216.90.105, Sub: 255....",,I dont think these are required by the ISP. 1....,,"<pad> static IP address IP: 64.216.90.105, Sub...",,
9,2000-10-16 06:44:00-07:00,phillip.allen@enron.com,zimam@enron.com,FW: fixed forward or other Collar floor gas pr...,"['', '---------------------- Forwarded by Phil...",,(),"((Phillip, K, Allen, /, HOU), (10/16/2000), (0...",---------------------- Forwarded by Phillip K...,"{0: ['forwarded', 'phillip', 'k', 'allen', 'ho...","To do', '> so I need a corresponding term gas ...",,"For 6-8 hours a day Microturbine run time:, > ...",,"Buck Buckner, P.E MBA, Manager, Busine

In [34]:

nlp = spacy.load('en_core_web_lg')
id=11
doc1 = nlp(df_emails['body_clean'][id])
doc2 = nlp(df_emails['summary_spacy'][id])

# Get the similarity of doc1 and doc2
similarity = doc1.similarity(doc2)
print(similarity)

0.8451136482559746


# SPACY - Summary (Abstractive / Word Embedding)

In [35]:
#!pip install spacy-transformers

In [36]:
# https://spacy.io/usage/embeddings-transformers
# https://spacy.io/api/transformer




In [37]:
# https://manusidtech.com/index.php/2021/08/14/complete-nlp-web-app-using-spacy-and-hugging-face-transformers/
#!pip install datasets transformers[sentencepiece]
#text = str(df_emails['body'][32])
#text="""The human coronavirus was first diagnosed in 1965 by Tyrrell and Bynoe from the respiratory tract sample of an adult with a common cold cultured on human embryonic trachea.1 Naming the virus is based on its crown-like appearance on its surface.2 Coronaviruses (CoVs) are a large family of viruses belonging to the Nidovirales order, which includes Coronaviridae, Arteriviridae, and Roniviridae families.3 Coronavirus contains an RNA genome and belongs to the Coronaviridae family.4 This virus is further subdivided into four groups, ie, the α, β, γ, and δ coronaviruses.5 α- and β-coronavirus can infect mammals, while γ- and δ- coronavirus tend to infect birds.6 Coronavirus in humans causes a range of disorders, from mild respiratory tract infections, such as the common cold to lethal infections, such as the severe acute respiratory syndrome (SARS), Middle East respiratory syndrome (MERS) and Coronavirus disease 2019 (COVID-19). The coronavirus first appeared in the form of severe acute respiratory syndrome coronavirus (SARS-CoV) in Guangdong province, China, in 20027 followed by Middle East respiratory syndrome coronavirus (MERS-CoV) isolated from the sputum of a 60-year-old man who presented symptoms of acute pneumonia and subsequent renal failure in Saudi Arabia in 2012.8 In December 2019, a β-coronavirus was discovered in Wuhan, China. The World Health Organization (WHO) has named the new disease as Coronavirus disease 2019 (COVID-19), and Coronavirus Study Group (CSG) of the International Committee has named it as SARS-CoV-2.9,10 Based on the results of sequencing and evolutionary analysis of the viral genome, bats appear to be responsible for transmitting the virus to humans"""
#print(text)

#print(summarize_abstractive (text))
#print(summarize_spacy (text))
#print()

A voir

In [38]:
# https://www.machinelearningplus.com/nlp/text-summarization-approaches-nlp-example/
# https://towardsdatascience.com/abstractive-summarization-using-pytorch-f5063e67510
# https://www.analyticsvidhya.com/blog/2021/10/text-summarization-using-the-conventional-hugging-face-transformer-and-cosine-similarity/


In [39]:
#!python -m spacy download en_core_web_lg

# Calcul similarités des résumés

In [40]:
spacy.load('en_core_web_lg')

In [41]:
nlp = spacy.load('en_core_web_lg')

In [43]:
print("Traitement en cours pour ",fin, "records")

for id in range(deb,fin):
  print("ID:",id)
  # Calcul de similarité avec texte initial nettoyé
  doc1 = nlp(df_emails['body_clean'][id])
  max_sim = 0
  doc2 = nlp(df_emails['summary_TFIDF'][id])
  if doc2 != "":
    sim = doc1.similarity(doc2)
    df_emails['summary_TFIDF_sim'][id] = sim
    max_sim = sim

  doc2 = nlp(df_emails['summary_spacy'][id])
  if doc2 != "":
    sim = doc1.similarity(doc2)
    df_emails['summary_spacy_sim'][id] = sim
    if sim > max_sim:
      max_sim = sim

  doc2 = nlp(df_emails['summary_T5'][id])
  if doc2 != "Erreur":
    sim = doc1.similarity(doc2)
    df_emails['summary_T5_sim'][id] = sim
    if sim > max_sim:
      max_sim = sim

  doc2 = nlp(df_emails['summary_BART'][id])
  if doc2 != "Erreur":
    sim = doc1.similarity(doc2)
    df_emails['summary_BART'][id] = sim
    if sim > max_sim:
      max_sim = sim

  print("Meilleur score : ",max_sim)
  df_emails['best_sim'][id] = max_sim


Traitement en cours pour  101 records
ID: 1
Meilleur score :  0.9699830412039898
ID: 2
Meilleur score :  1.0
ID: 3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


Meilleur score :  1.0
ID: 4
Meilleur score :  1.0
ID: 5
Meilleur score :  1.0
ID: 6
Meilleur score :  1.0
ID: 7
Meilleur score :  1.0
ID: 8
Meilleur score :  1.0
ID: 9
Meilleur score :  0.9618100370395811
ID: 10
Meilleur score :  1.0
ID: 11
Meilleur score :  1.0
ID: 12


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


Meilleur score :  0.987251737127064
ID: 13
Meilleur score :  0.9872473790496183
ID: 14
Meilleur score :  1.0
ID: 15
Meilleur score :  1.0
ID: 16
Meilleur score :  1.0
ID: 17
Meilleur score :  1.0
ID: 18
Meilleur score :  0.9470007976909709
ID: 19
Meilleur score :  1.0
ID: 20
Meilleur score :  1.0
ID: 21
Meilleur score :  1.0
ID: 22
Meilleur score :  1.0
ID: 23
Meilleur score :  0.983569588437822
ID: 24
Meilleur score :  0.9881647220517387
ID: 25
Meilleur score :  0.986456887318194
ID: 26
Meilleur score :  0.9864564996888571
ID: 27
Meilleur score :  1.0
ID: 28
Meilleur score :  0.9852105159826147
ID: 29
Meilleur score :  1.0
ID: 30
Meilleur score :  1.0
ID: 31
Meilleur score :  1.0
ID: 32
Meilleur score :  0.9862549300910006
ID: 33
Meilleur score :  1.0
ID: 34
Meilleur score :  1.0
ID: 35
Meilleur score :  1.0
ID: 36
Meilleur score :  1.0
ID: 37
Meilleur score :  1.0
ID: 38
Meilleur score :  1.0
ID: 39
Meilleur score :  1.0
ID: 40
Meilleur score :  0.9802432263527312
ID: 41
Meilleur sco

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


Meilleur score :  0.9546171716011248
ID: 51
Meilleur score :  0.9833239189568894
ID: 52
Meilleur score :  0.9954963001616991
ID: 53
Meilleur score :  1.0
ID: 54
Meilleur score :  1.0
ID: 55
Meilleur score :  1.0
ID: 56
Meilleur score :  0.9694593789634784
ID: 57
Meilleur score :  0.9833114380745906
ID: 58
Meilleur score :  1.0
ID: 59
Meilleur score :  1.0
ID: 60
Meilleur score :  0.942093131739083
ID: 61
Meilleur score :  0.9661263236709299
ID: 62
Meilleur score :  1.0
ID: 63
Meilleur score :  1.0
ID: 64
Meilleur score :  1.0
ID: 65
Meilleur score :  0.930088293204881
ID: 66
Meilleur score :  1.0
ID: 67
Meilleur score :  1.0
ID: 68
Meilleur score :  1.0
ID: 69
Meilleur score :  1.0
ID: 70
Meilleur score :  0.9099145805746778
ID: 71
Meilleur score :  0.9156918449071875
ID: 72
Meilleur score :  1.0
ID: 73
Meilleur score :  1.0
ID: 74
Meilleur score :  1.0
ID: 75
Meilleur score :  1.0
ID: 76
Meilleur score :  1.0
ID: 77
Meilleur score :  1.0
ID: 78
Meilleur score :  1.0
ID: 79
Meilleur sc

# Sauvegarde du fichier final en .csv et .xls

In [45]:
# SAUVEGARDE en csv du résultat.
file_path = "/content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/"
file_name = "enron_unique_output"

#fin = df_emails.shape[0]
#fin = 200

# to .CSV
file_extension = ".csv"
file = file_path+file_name+file_extension
print("File : ",file)
df_emails[deb:fin].to_csv(file, encoding='utf-8', index=False)
# to XLS
file_extension = ".xls"
file = file_path+file_name+file_extension
df_emails[deb:fin].to_excel(file, encoding='utf-8', index=False)



File :  /content/drive/MyDrive/Datascientest/Projet PY_email Datascientest/Data/Enron Cleaned Data/enron_unique_output.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  app.launch_new_instance()


# Archived

# Clusterisation des mails

In [ ]:
# pour les classifier par theme(s) en fonction des NER filtrés (en enlevant les dates par exemple)
# https://colab.research.google.com/drive/1BlK5Jxn7BQQwQhYhD-q6nGD9_wVbIYuq?usp=sharing#scrollTo=HdICm_4AyeOX

In [ ]:
# To Do ?
# identifier réponses et forward avec ou sans commentaires